In [1]:
import IPython
import reboting
from reboting import CouldntSaveDataException
from reboting import CsvDownloadAndParsingException
from reboting import CouldNotDownloadFileException
from reboting import CouldntCreateVisualException
from reboting import DataTooBigException
from reboting import NoVisualsPossible
import requests
import pandas as pd
import time

go over all queries for each bundesland

- for every item ask the bot with url
- and then analyze the db

In [ ]:
stats = { 'total' : {
    'total' : 0,
    'success' : 0
}}
config = {
    'vienna' : '&l=http://sws.geonames.org/2761369/&l=http://sws.geonames.org/2761367/&l=http://sws.geonames.org/2761333/&l=http://sws.geonames.org/2761368/',
    'styria' : '&l=http://sws.geonames.org/2764581/',
    'lower_austria' : '&l=http://sws.geonames.org/2770542/',
    'upper_austria' : '&l=http://sws.geonames.org/2769848/',
    'tirol' : '&l=http://sws.geonames.org/2763587/&l=http://sws.geonames.org/2763586/&l=http://sws.geonames.org/2763588/',
    'vorarlberg' : '&l=http://sws.geonames.org/2762300/',
    'carinthia' : '&l=http://sws.geonames.org/2774686/',
    'burgenland' : '&l=http://sws.geonames.org/6955321/&l=http://sws.geonames.org/2781194/',
    'salzburg' : '&l=http://sws.geonames.org/2766824/&l=http://sws.geonames.org/2766818/&l=http://sws.geonames.org/2766823/&l=http://sws.geonames.org/7871495/'
}
for key, value in config.items():
    #print(key+" val: "+value)
    offset=0
    url ='http://data.wu.ac.at/odgraph/locationsearch?a=1'+value+'&offset='+str(offset)+'&limit=1'
    r = requests.get(url)
    resp = r.json()  
    stats[key]={ 'total': resp['total'], 'success' : 0}
    stats['total']['total'] = stats['total']['total']+resp['total']
    #lets start the for loop again
    #resp['total'] resp['total']
    for i in range(0,resp['total']):        
        offset=i        
        url ='http://data.wu.ac.at/odgraph/locationsearch?a=1'+value+'&offset='+str(offset)+'&limit=1'
        time.sleep(3)
        try: 
            r = requests.get(url)
            resp = r.json()  
            #hardcoded userid
            user_id='10213970072671093'
            searchresult = resp['results'][0]           
            geolocations = set(['http://sws.geonames.org/2782113/'])
            if searchresult.get('entities',None) is not None:
                #get the array
                for entity in searchresult['entities']:
                    if(entity!=""):
                        #print("adding entity: "+entity)
                        geolocations.add(entity)              
            if searchresult.get('locations',None) is not None:
                #get the array
                for entity in searchresult['locations']:
                    if(entity!=""):
                        #print("adding entity from location: "+entity)
                        geolocations.add(entity)
            data_desc = {
                    "name" : "No name available",
                    "description" : "No description available",
                    "publisher" : "no publisher available",
                    "portal" : "No Portal available",
                    "url": searchresult['url'],
                    "user_id" : user_id,
                    "geolocations" : geolocations                    
            }
            #"temporal_end": "2014-01-09", 
            #"temporal_start": "2014-01-09", 
            if(searchresult.get('temporal_end',None) is not None and searchresult.get('temporal_start',None) is not None):
                print("adding temporal parts")
                data_desc['temporal_end'] = searchresult['temporal_end']
                data_desc['temporal_start'] = searchresult['temporal_start']
            if searchresult.get('dataset',None) is not None:
                if searchresult['dataset'].get('dataset_name',None) is not None:
                    data_desc['name'] = searchresult['dataset']['dataset_name']
                if searchresult['dataset'].get('dataset_description',None) is not None:
                    data_desc['description'] = searchresult['dataset']['dataset_description']
                if searchresult['dataset'].get('publisher',None) is not None:
                    data_desc['publisher'] = searchresult['dataset']['publisher']
            if searchresult.get('portal',None) is not None:
                data_desc['portal'] = searchresult['portal']            
            try:
                response = reboting.importAsNew(data_desc = data_desc)
                #url = 'https://doh.23degrees.io/embed/'+slug
                print(response)
                if(response>0):                    
                    stats[key]['success'] = stats[key]['success']+1
                    stats['total']['success'] = stats['total']['success']+1                    
            except CouldntSaveDataException as e:
                print("ERROR: couldnt save data on 23 degree server")
            except CsvDownloadAndParsingException as e:
                print("ERROR: something went wrong when downloading the file")
            except CouldNotDownloadFileException as e:
                print("ERROR: couldnt download file")
            except CouldntCreateVisualException as e:
                print("ERROR: couldnt create visual")
            except DataTooBigException as e:
                print("ERROR: too many lines: "+str(e))   
            except NoVisualsPossible as e:
                print("ERROR: no visual possible: "+str(e))
            print(str(i)+" success: "+str(stats['total']['success']))
        except Exception as e:
            #raise Exception(e)
            print("new error: "+str(e))

Error from mdc for data: https://www.wien.gv.at/politik/wahlen/ogd/bv161_99999999_9999_spr.csv
{'action': 'fields - get_fields_object', 'client_message': 'Error while constructing the fields-object for MetaData', 'message': 'None', 'status': 'error'}
0
0 success: 0
Error from mdc for data: https://www.wien.gv.at/statistik/ogd/vie_209.csv
{'action': 'dataPipeline - complete_integration - array_of_vizzes is empty', 'client_message': 'An error occurred in the data-integration process.', 'message': 'array_of_vizzes is empty - no valid vizzes.', 'status': 'error'}
0
1 success: 0
Error from mdc for data: https://www.wien.gv.at/politik/wahlen/ogd/nr081_99999999_9999_wvb.csv
{'action': 'helpers - geo_ref - checkPotentialIsos', 'client_message': 'checkPotentialLabels done found: 0', 'message': 'No matches found!', 'status': 'error'}
0
2 success: 0
Error from mdc for data: https://www.wien.gv.at/politik/wahlen/ogd/nr131_99999999_9999_wvb.csv
{'action': 'helpers - geo_ref - checkPotentialIsos', '

In [ ]:
stats